In [2]:
from ortools.linear_solver import pywraplp

def read_input():

    N, M, K = map(int, input().split())
    
    parcel_quantities = list(map(int, input().split()))
    
    taxi_capacities = list(map(int, input().split()))
    
    distance_matrix = []
    for _ in range(2 * N + 2 * M + 1):
        row = list(map(int, input().split()))
        distance_matrix.append(row)
        
    return N, M, K, parcel_quantities, taxi_capacities, distance_matrix

def create_variables(solver, N, M, K):
    """
    Create optimization variables
    """
    # X[i,j,k] = 1 if taxi k travels from i to j
    X = {}
    for i in range(2 * N + 2 * M + 1):
        for j in range(2 * N + 2 * M + 1):
            for k in range(K):
                X[i, j, k] = solver.BoolVar(f'X[{i},{j},{k}]')
    
    # Y[i,k] = 1 if passenger i is served by taxi k
    Y = {}
    for i in range(N):
        for k in range(K):
            Y[i, k] = solver.BoolVar(f'Y[{i},{k}]')
    
    # Z[i,k] = 1 if parcel i is carried by taxi k
    Z = {}
    for i in range(M):
        for k in range(K):
            Z[i, k] = solver.BoolVar(f'Z[{i},{k}]')
            
    # T[k] = total distance traveled by taxi k
    T = [solver.IntVar(0, solver.infinity(), f'T[{k}]') for k in range(K)]
    
    # D = maximum distance among all taxis
    D = solver.IntVar(0, solver.infinity(), 'D')
    
    return X, Y, Z, T, D

def add_constraints(solver, N, M, K, X, Y, Z, T, D, parcel_quantities, taxi_capacities, distance_matrix):
    #@@

    for k in range(K):
        solver.Add(T[k] <= D)
    #@@
    for i in range(N):
        solver.Add(sum(Y[i, k] for k in range(K)) == 1)
        # Link passenger pickup and dropoff
        for k in range(K):
            solver.Add(X[i+1, i+1+ N + M, k] == Y[i, k])
    
    #@@
    for k in range(K):
        solver.Add(sum(parcel_quantities[i] * Z[i, k] for i in range(M)) <= taxi_capacities[k])
    
    #@@
    for k in range(K):
        solver.Add(sum(X[0, j, k] for j in range(1, 2 * N + 2 * M + 1)) == 1)
        solver.Add(sum(X[i, 0, k] for i in range(1, 2 * N + 2 * M + 1)) == 1)
        
    # Each parcel must be carried by exactly one taxi
    for i in range(M):
        solver.Add(sum(Z[i, k] for k in range(K)) == 1)
        

    
    # Flow conservation constraints
    for k in range(K):
        for j in range(2 * N + 2 * M + 1):
            solver.Add(sum(X[i, j, k] for i in range(2 * N + 2 * M + 1) if i != j) ==
                      sum(X[j, i, k] for i in range(2 * N + 2 * M + 1) if i != j))
    
    # Calculate total distance for each taxi
    for k in range(K):
        solver.Add(T[k] == sum(distance_matrix[i][j] * X[i, j, k]
                              for i in range(2 * N + 2 * M + 1)
                              for j in range(2 * N + 2 * M + 1)))
    
# Ensure pickup before delivery for parcels
    for i in range(M):
        for k in range(K):
            # Ensure that taxi k picks up the parcel at node N + i + 1 before it can deliver it to node N + M + i + 1
            solver.Add(sum(X[j, N + i + 1, k] for j in range(2 * N + 2 * M + 1)) >=
                      sum(X[N + i + 1, j, k] for j in range(2 * N + 2 * M + 1)))

            # Ensure that the parcel is delivered at node N + M + i + 1 after being picked up
            solver.Add(sum(X[j, N + M + i + 1, k] for j in range(2 * N + 2 * M + 1)) >=
                      sum(X[N + i + 1, j, k] for j in range(2 * N + 2 * M + 1)))


            
def extract_routes(X, K, N, M):
    """
    Extract the route for each taxi from the solution
    """
    routes = []
    for k in range(K):
        route = []
        current_point = 0
        visited = set([0])
        
        while True:
            found_next = False
            for j in range(2 * N + 2 * M + 1):
                if X[current_point, j, k].solution_value() > 0.5 and j not in visited:
                    route.append(j)
                    visited.add(j)
                    current_point = j
                    found_next = True
                    break
            if not found_next:
                break
        routes.append(route)
    return routes

def main():
    # Create solver
    solver = pywraplp.Solver.CreateSolver('SCIP')
    
    # Read input
    N = 3
    M = 3
    K = 2
    parcel_quantities = [8, 4, 5]
    taxi_capacities = [16, 16]
    distance_matrix = [
    [0, 8, 7, 9, 6, 5, 11, 6, 11, 12, 12, 12, 13],
    [8, 0, 4, 1, 2, 8, 5, 13, 19, 12, 4, 8, 9],
    [7, 4, 0, 3, 3, 8, 4, 12, 15, 8, 5, 6, 7],
    [9, 1, 3, 0, 3, 9, 4, 14, 19, 11, 3, 7, 8],
    [6, 2, 3, 3, 0, 6, 6, 11, 17, 11, 6, 9, 10],
    [5, 8, 8, 9, 6, 0, 12, 5, 16, 15, 12, 15, 15],
    [11, 5, 4, 4, 6, 12, 0, 16, 18, 7, 4, 3, 4],
    [6, 13, 12, 14, 11, 5, 16, 0, 15, 18, 17, 18, 19],
    [11, 19, 15, 19, 17, 16, 18, 15, 0, 13, 21, 17, 17],
    [12, 12, 8, 11, 11, 15, 7, 18, 13, 0, 11, 5, 4],
    [12, 4, 5, 3, 6, 12, 4, 17, 21, 11, 0, 7, 8],
    [12, 8, 6, 7, 9, 15, 3, 18, 17, 5, 7, 0, 1],
    [13, 9, 7, 8, 10, 15, 4, 19, 17, 4, 8, 1, 0]
    ]
    for i in range(2 * N + 2 * M + 1):
        for j in range(2 * N + 2 * M + 1):
            if i==j:
                distance_matrix[i][j]= 0
            
    # Create variables
    X, Y, Z, T, D = create_variables(solver, N, M, K)
    
    # Set objective: minimize maximum distance
    solver.Minimize(D)
    
    # Add constraints
    add_constraints(solver, N, M, K, X, Y, Z, T, D, parcel_quantities, taxi_capacities, distance_matrix)
    
    # Solve the problem
    status = solver.Solve()
    
    # Output results
    if status == pywraplp.Solver.OPTIMAL:
        print(K)
        routes = extract_routes(X, K, N, M)
        for route in routes:
            print(len(route) + 2)  # +2 for start and end depot
            print(f"0 {' '.join(map(str, route))} 0")
    else:
        print('No optimal solution found.')
main()

2
5
0 4 1 7 0
4
0 2 8 0


In [ ]:
from ortools.linear_solver import pywraplp

def read_input():

    # Read first line containing problem dimensions
    N, M, K = map(int, input().split())
    
    # Read parcel quantities
    parcel_quantities = list(map(int, input().split()))
    
    # Read taxi capacities  
    taxi_capacities = list(map(int, input().split()))
    
    # Read distance matrix
    distance_matrix = []
    for _ in range(2 * N + 2 * M + 1):
        row = list(map(int, input().split()))
        distance_matrix.append(row)
        
    return N, M, K, parcel_quantities, taxi_capacities, distance_matrix

def create_variables(solver, N, M, K):
    """
    Create optimization variables
    """
    # X[i,j,k] = 1 if taxi k travels from i to j
    X = {}
    for i in range(2 * N + 2 * M + 1):
        for j in range(2 * N + 2 * M + 1):
            for k in range(K):
                X[i, j, k] = solver.BoolVar(f'X[{i},{j},{k}]')
    
    # Y[i,k] = 1 if passenger i is served by taxi k
    Y = {}
    for i in range(N):
        for k in range(K):
            Y[i, k] = solver.BoolVar(f'Y[{i},{k}]')
    
    # Z[i,k] = 1 if parcel i is carried by taxi k
    Z = {}
    for i in range(M):
        for k in range(K):
            Z[i, k] = solver.BoolVar(f'Z[{i},{k}]')
            
    # T[k] = total distance traveled by taxi k
    T = [solver.NumVar(0, solver.infinity(), f'T[{k}]') for k in range(K)]
    
    # D = maximum distance among all taxis
    D = solver.NumVar(0, solver.infinity(), 'D')
    
    return X, Y, Z, T, D

def add_constraints(solver, N, M, K, X, Y, Z, T, D, parcel_quantities, taxi_capacities, distance_matrix):

    # Maximum distance constraint
    for k in range(K):
        solver.Add(T[k] <= D)
    
    # Each passenger must be served by exactly one taxi
    for i in range(N):
        solver.Add(sum(Y[i, k] for k in range(K)) == 1)
        # Link passenger pickup and dropoff
        for k in range(K):
            solver.Add(X[i, i + N + M, k] == Y[i, k])
    
    # Taxi capacity constraints for parcels
    for k in range(K):
        solver.Add(sum(parcel_quantities[i] * Z[i, k] for i in range(M)) <= taxi_capacities[k])
    
    # Each parcel must be carried by exactly one taxi
    for i in range(M):
        solver.Add(sum(Z[i, k] for k in range(K)) == 1)
        # Link parcel pickup and delivery
        for k in range(K):
            solver.Add(X[N + i, N + M + i, k] == Z[i, k])
    
    # All taxis must start and end at depot
    for k in range(K):
        solver.Add(sum(X[0, j, k] for j in range(1, 2 * N + 2 * M + 1)) == 1)
        solver.Add(sum(X[i, 0, k] for i in range(1, 2 * N + 2 * M + 1)) == 1)
    
    # Flow conservation constraints
    for k in range(K):
        for j in range(2 * N + 2 * M + 1):
            solver.Add(sum(X[i, j, k] for i in range(2 * N + 2 * M + 1) if i != j) ==
                      sum(X[j, i, k] for i in range(2 * N + 2 * M + 1) if i != j))
    
    # Calculate total distance for each taxi
    for k in range(K):
        solver.Add(T[k] == sum(distance_matrix[i][j] * X[i, j, k]
                              for i in range(2 * N + 2 * M + 1)
                              for j in range(2 * N + 2 * M + 1)))
    
    # Ensure pickup before delivery for parcels
    for i in range(M):
        for k in range(K):
            solver.Add(sum(j * X[N + i, j, k] for j in range(2 * N + 2 * M + 1)) >=
                      sum(j * X[j, N + M + i, k] for j in range(2 * N + 2 * M + 1)))

def extract_routes(X, K, N, M):
    """
    Extract the route for each taxi from the solution
    """
    routes = []
    for k in range(K):
        route = []
        current_point = 0
        visited = set([0])
        
        while True:
            found_next = False
            for j in range(2 * N + 2 * M + 1):
                if X[current_point, j, k].solution_value() > 0.5 and j not in visited:
                    route.append(j)
                    visited.add(j)
                    current_point = j
                    found_next = True
                    break
            if not found_next:
                break
        routes.append(route)
    return routes

def main():
    # Create solver
    solver = pywraplp.Solver.CreateSolver('SCIP')
    
    # Read input
    N, M, K, parcel_quantities, taxi_capacities, distance_matrix = read_input()
    
    # Create variables
    X, Y, Z, T, D = create_variables(solver, N, M, K)
    
    # Set objective: minimize maximum distance
    solver.Minimize(D)
    
    # Add constraints
    add_constraints(solver, N, M, K, X, Y, Z, T, D, parcel_quantities, taxi_capacities, distance_matrix)
    
    # Solve the problem
    status = solver.Solve()
    
    # Output results
    if status == pywraplp.Solver.OPTIMAL:
        print(K)
        routes = extract_routes(X, K, N, M)
        for route in routes:
            print(len(route) + 2)  # +2 for start and end depot
            print(f"0 {' '.join(map(str, route))} 0")
    else:
        print('No optimal solution found.')

if _name_ == "_main_":
    main()